# Save courses to RDF

## Read data from Firestore

In [1]:
# init
from os import environ
environ['FIRESTORE_EMULATOR_HOST'] = 'localhost:8080'
environ['FIREBASE_AUTH_EMULATOR_HOST'] = 'localhost:8083'
environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../serviceAccountKey.json'

from firebase_admin import firestore, initialize_app, credentials
cred = credentials.ApplicationDefault()
initialize_app(cred)
db = firestore.client()

In [9]:
# get a program
siit_ref = db.collection(u'programs').document(u'siit')
program = siit_ref.get().to_dict()
program['naziv']

'softversko inzenjerstvo i informacione tehnologije'

In [17]:
# get courses
courses_snapshot = siit_ref.collection(u'courses').stream()
courses = [course.to_dict() for course in courses_snapshot]
courses[0]['naziv']

'Završni rad - izrada i odbrana'

## Write courses to rdf

Simple rdflib example

In [33]:
from rdflib import *

# create Bob & Linda
bob = URIRef('http://example.org/people/Bob')
linda = BNode()

# create Literals
name = Literal('Bob')
age = Literal(24)
height = Literal(180)

# create graph & add namespace
g = Graph()
g.bind('foaf', FOAF)

# add Bob & Linda to the graph
g.add((bob, RDF.type, FOAF.Person))
g.set((bob, FOAF.name, name))
g.set((bob, FOAF.age, age))
g.add((bob, FOAF.knows, linda))
g.add((linda, RDF.type, FOAF.Person))
g.set((linda, FOAF.name, Literal('LINDA')))

# write graph to file
g.serialize(destination='program.ttl')

# print graph, print values in graph
print(g.serialize(), end='\n-----------\n')
print(g.value(bob, FOAF.age))

# remove all triples about bob
g.remove((bob, None, None)) 
print(g.serialize(), end='\n-----------\n')

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/people/Bob> a foaf:Person ;
    foaf:age 24 ;
    foaf:knows [ a foaf:Person ;
            foaf:name "LINDA" ] ;
    foaf:name "Bob" .


-----------
24
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

[] a foaf:Person ;
    foaf:name "LINDA" .


-----------


In [57]:
g = Graph()
g.bind('dc', DC)
professors = courses[1]['izvodjaciNastave']
print(professors)

for course in courses[:2]:
    course_node = URIRef(course['url'])

    g.add((course_node, RDF.type, OWL.Ontology))
    g.add((course_node, DC.title, Literal(course['naziv'])))
    g.add((course_node, DC.description, Literal(course['sadrzaj'])))


print(g.serialize(format='turtle'))
# print(g.serialize(format='turtle', destination='courses.ttl'))


[{'zvanje': 'Redovni profesor', 'punoIme': 'Grubić-Nešić dr Leposava', 'vidNastave': 'Predavanja'}, {'zvanje': 'Vanredni profesor', 'vidNastave': 'Predavanja', 'punoIme': 'Katić dr Ivana'}, {'zvanje': 'Vanredni profesor', 'punoIme': 'Katić dr Ivana', 'vidNastave': 'Auditorne vežbe'}]
@prefix dc: <http://purl.org/dc/elements/1.1/> .

<http://www.ftn.uns.ac.rs/n636239540/zavrsni-rad---izrada-i-odbrana> a <http://www.w3.org/2002/07/owl#Ontology> ;
    dc:description "Formira se pojedinačno u skladu sa potrebama i oblašću koja je obuhvaćena zadatom temom završnog rada. Student u dogovoru sa mentorom sačinjava završni rad u pismenoj formi u skladu sa predviđenim standardima Fakulteta tehničkih nauka. Student priprema i brani pismeni završni rad javno u dogovoru sa mentorom i u skladu sa predviđenim standardima. Student proučava stručnu literaturu, stručne i diplomske radove studenata koji se bave sličnom tematikom, vrši analize u cilju iznalaženja rešenja konkretnog zadatka koji je definisa